In [1]:
import pandas as pd
from scrape_flight_crl import parse_flights, get_arrivals, get_departures, URL

In [2]:
df = parse_flights(url="backup/url_2017-11-19T22.html")

In [3]:
arrivals, departures = get_arrivals(df), get_departures(df)

In [4]:
arrivals.tail()

Flightnr. Depart Arrive      Start        End   Mon.   Tue.  \
connection    #                                                               
Vilnius->CRL  1    FR6459  21:10  22:45 2017-12-28 2017-12-28  False  False   
              2    FR6459  21:10  22:45 2018-04-01 2018-03-22  False  False   
Zaragoza->CRL 0    FR1383  10:25  12:30 2017-12-19 2017-12-19  False   True   
              1    FR1383  10:25  12:30 2018-02-01 2018-02-01  False   True   
              2    FR1383  19:25  21:30 2017-01-11 2018-03-24  False  False   

                  Wed.   Thu.   Fri.   Sat.   Sun.  
connection    #                                     
Vilnius->CRL  1  False   True  False  False  False  
              2  False   True  False  False   True  
Zaragoza->CRL 0  False  False  False  False  False  
              1  False  False  False  False  False  
              2   True  False  False   True  False

In [5]:
arrivals.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 495 entries, (Agadir->CRL, 0) to (Zaragoza->CRL, 2)
Data columns (total 12 columns):
Flightnr.    495 non-null object
Depart       495 non-null object
Arrive       495 non-null object
Start        495 non-null datetime64[ns]
End          495 non-null datetime64[ns]
Mon.         495 non-null bool
Tue.         495 non-null object
Wed.         495 non-null object
Thu.         495 non-null object
Fri.         495 non-null object
Sat.         495 non-null object
Sun.         495 non-null object
dtypes: bool(1), datetime64[ns](2), object(9)
memory usage: 46.1+ KB


In [6]:
arrivals_flat = arrivals.reset_index()
arrivals_flat.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,Mon.,Tue.,Wed.,Thu.,Fri.,Sat.,Sun.
490,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,False,False,False,True,False,False,False
491,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,False,False,False,True,False,False,True
492,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,False,True,False,False,False,False,False
493,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,False,True,False,False,False,False,False
494,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,False,False,True,False,False,True,False


In [7]:
arrivals_flat.columns[7:]

Index(['Mon.', 'Tue.', 'Wed.', 'Thu.', 'Fri.', 'Sat.', 'Sun.'], dtype='object')

Rename columns to make easy the sorting later

In [8]:
d = dict(zip(arrivals_flat.columns[7:], range(7)))
d

{'Fri.': 4, 'Mon.': 0, 'Sat.': 5, 'Sun.': 6, 'Thu.': 3, 'Tue.': 1, 'Wed.': 2}

In [9]:
arrivals_flat.rename(columns=d, inplace=True)
arrivals_flat.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,0,1,2,3,4,5,6
490,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,False,False,False,True,False,False,False
491,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,False,False,False,True,False,False,True
492,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,False,True,False,False,False,False,False
493,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,False,True,False,False,False,False,False
494,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,False,False,True,False,False,True,False


In [10]:
wd = arrivals_flat.melt(id_vars=arrivals_flat.columns[:7], value_vars=arrivals_flat.columns[7:], var_name="weekday")

In [11]:
wd.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday,value
3460,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,6,False
3461,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,6,True
3462,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,6,False
3463,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,6,False
3464,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,6,False


In [12]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3465 entries, 0 to 3464
Data columns (total 9 columns):
connection    3465 non-null object
#             3465 non-null int64
Flightnr.     3465 non-null object
Depart        3465 non-null object
Arrive        3465 non-null object
Start         3465 non-null datetime64[ns]
End           3465 non-null datetime64[ns]
weekday       3465 non-null object
value         3465 non-null object
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 243.7+ KB


Remove rows with the value of the weekday False. They carry no information

In [13]:
wd_true = wd[wd.loc[:,"value"]].drop(columns="value")

In [14]:
wd_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 3461
Data columns (total 8 columns):
connection    1100 non-null object
#             1100 non-null int64
Flightnr.     1100 non-null object
Depart        1100 non-null object
Arrive        1100 non-null object
Start         1100 non-null datetime64[ns]
End           1100 non-null datetime64[ns]
weekday       1100 non-null object
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 77.3+ KB


In [15]:
wd_true_sorted = wd_true.sort_values(["connection", "weekday"]).reset_index(drop=True)
wd_true_sorted.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday
1095,Warsaw->CRL,6,W61381,18:40,20:40,2017-10-29,2017-12-15,6
1096,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,1
1097,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,1
1098,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,2
1099,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,5


Sort by weeday and put back day names

In [16]:
wd_true_sorted['weekday'] = wd_true_sorted['weekday'].replace({v: k for k, v in d.items()})

In [17]:
wd_true_sorted.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday
1095,Warsaw->CRL,6,W61381,18:40,20:40,2017-10-29,2017-12-15,Sun.
1096,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,Tue.
1097,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,Tue.
1098,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,Wed.
1099,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,Sat.


In [18]:
arrivals_weekdays = wd_true_sorted.set_index(["connection", "weekday"])
arrivals_weekdays.tail()

# Flightnr. Depart Arrive      Start        End
connection    weekday                                                 
Warsaw->CRL   Sun.     6    W61381  18:40  20:40 2017-10-29 2017-12-15
Zaragoza->CRL Tue.     0    FR1383  10:25  12:30 2017-12-19 2017-12-19
              Tue.     1    FR1383  10:25  12:30 2018-02-01 2018-02-01
              Wed.     2    FR1383  19:25  21:30 2017-01-11 2018-03-24
              Sat.     2    FR1383  19:25  21:30 2017-01-11 2018-03-24

In [19]:
arrivals_weekdays.reset_index().tail()

,connection,weekday,#,Flightnr.,Depart,Arrive,Start,End
1095,Warsaw->CRL,Sun.,6,W61381,18:40,20:40,2017-10-29,2017-12-15
1096,Zaragoza->CRL,Tue.,0,FR1383,10:25,12:30,2017-12-19,2017-12-19
1097,Zaragoza->CRL,Tue.,1,FR1383,10:25,12:30,2018-02-01,2018-02-01
1098,Zaragoza->CRL,Wed.,2,FR1383,19:25,21:30,2017-01-11,2018-03-24
1099,Zaragoza->CRL,Sat.,2,FR1383,19:25,21:30,2017-01-11,2018-03-24


In [20]:
def reshape_with_weekday_column(df):
    df_flat = df.reset_index()
    # Find first weekday column
    first_day_col = list(df_flat.columns).index("Mon.")
    # Rename columns to make easy the sorting later
    d = dict(zip(df_flat.columns[first_day_col:], range(7)))
    df_flat.rename(columns=d, inplace=True)
    # Reshape the dataframe creating a column 'weekday'
    wd = df_flat.melt(id_vars=df_flat.columns[:7], value_vars=df_flat.columns[7:], var_name="weekday")
    # Take only good rows and sort
    wd_true = wd[wd.loc[:,"value"]].drop(columns="value")
    wd_true_sorted = wd_true.sort_values(["connection", "weekday"]).reset_index(drop=True)
    # Restore day names
    wd_true_sorted['weekday'] = wd_true_sorted['weekday'].replace({v: k for k, v in d.items()})
    df_weekdays = wd_true_sorted #.set_index(["connection", "weekday"])
    return df_weekdays

In [21]:
departures = reshape_with_weekday_column(departures)
departures.head()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday
0,CRL->Agadir,0,FR8172,06:30,09:20,2017-10-30,2017-12-22,Mon.
1,CRL->Agadir,4,FR8172,06:30,09:20,2018-08-01,2018-03-23,Mon.
2,CRL->Agadir,5,FR8172,08:00,10:50,2018-01-01,2018-01-01,Mon.
3,CRL->Agadir,1,FR8172,06:30,09:20,2017-12-19,2017-12-19,Tue.
4,CRL->Agadir,3,FR8172,06:30,09:20,2018-02-01,2018-02-01,Tue.


Some example of what can be done with these dataframes (see [this other notebook](Analyze flights.ipynb) for details)

In [22]:
departures.groupby("weekday").count()

,connection,#,Flightnr.,Depart,Arrive,Start,End
weekday,,,,,,,
Fri.,160,160,160,160,160,160,160
Mon.,165,165,165,165,165,165,165
Sat.,150,150,150,150,150,150,150
Sun.,178,178,178,178,178,178,178
Thu.,152,152,152,152,152,152,152
Tue.,155,155,155,155,155,155,155
Wed.,201,201,201,201,201,201,201


In [23]:
df = reshape_with_weekday_column(df)
df.head()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday
0,Agadir->CRL,0,FR8173,09:45,14:35,2017-10-30,2017-12-22,Mon.
1,Agadir->CRL,4,FR8173,09:45,14:35,2018-08-01,2018-03-23,Mon.
2,Agadir->CRL,5,FR8173,11:15,16:05,2018-01-01,2018-01-01,Mon.
3,Agadir->CRL,1,FR8173,09:45,14:35,2017-12-19,2017-12-19,Tue.
4,Agadir->CRL,3,FR8173,09:45,14:35,2018-02-01,2018-02-01,Tue.


In [24]:
arrivals_weekdays.groupby("weekday").count()

,#,Flightnr.,Depart,Arrive,Start,End
weekday,,,,,,
Fri.,149,149,149,149,149,149
Mon.,157,157,157,157,157,157
Sat.,146,146,146,146,146,146
Sun.,173,173,173,173,173,173
Thu.,147,147,147,147,147,147
Tue.,153,153,153,153,153,153
Wed.,175,175,175,175,175,175
